<a href="https://colab.research.google.com/github/smazumder05/my-chatgpt/blob/main/transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget "https://raw.githubusercontent.com/smazumder05/my-chatgpt/main/data/tiny-shakespear/input.txt"

In [ ]:
# ead the file to inspect its contents and print out the number of characters
with open("input.txt", 'r', encoding="utf-8") as f:
  text = f.read()

print("the number of charecters in the file:", len(text))
# print the first 2000 charecters in the file
print(text[:2000])

In [ ]:
# Number of unique characters in the file
uniqueChars = sorted(list(set(text)))
print(len(uniqueChars))
print(''.join(uniqueChars))

In [ ]:
#Basic tokenizer, there are other tokenizer such as SentencePiece or tikTokenizer
#create a mapping from charatecters to integers
stoi = {ch:i for i, ch in enumerate(uniqueChars)}
itos = {i:ch for i, ch in enumerate(uniqueChars)}
#encode takes a string and outputs a list of integers
encode = lambda s: [stoi[c] for c in s]
#decode takes a list of integers and outputs a string
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

In [ ]:
#Use the tokenizer to encode the entire text from time Shakespear
# using Pytorch library for torch data structure
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) #print the first 1000 characters

In [ ]:
# In order to train the model, we need to split the data into train and validation sets
n = int(0.9*len(data)) #Get the first 90% of the data
train_data = data[:n]
val_data = data[n:]

In [ ]:
# train the model with random text of maximum length also known as block size or context_size
# we use a context_size of 8
context_size = 8
train_data[:context_size+1]
#The transformer will learn to predict what char comes next, so it starts with 18 and predicts 47, if one gives it 18 47 it will predict 56 and so on

In [ ]:
x = train_data[:context_size] #first context_size characters
y = train_data[1:context_size+1] #next context_size + 1 characters, it is the target
for t in range(context_size):
  context = x[:t+1]
  target = y[t]
  print(f"when input is: {context} the target is: {target}")

In [ ]:
# Now create batch dimension for efficiency
torch.manual_seed(1337)
batch_size = 4 #how many will we process in parallel
block_size = 8 # maximum context length for the predictions

def get_batch(split):
  #generate a small batch of data of x inputs and y targets
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('___________________________')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b :t+1]
    target  = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target} ")
    